In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
import os
import xgboost as xgb
import catboost

warnings.filterwarnings(action='ignore')

In [61]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [62]:
train_df.head()

,Property_ID,room,layout_type,property_type,locality,price,area,furnish_type,bathroom,city,...,floor,pet_friendly,power_backup,washing_machine,air_conditioner,geyser/solar,security_deposit,CCTV/security,lift,neighbourhood
0,42208,3,BHK,Independent House,Palavakkam,33624,1312,Furnished,2,Chennai,...,1,1,0,0,1,0,302616,0,0,300
1,90879,1,BHK,Apartment,Manikonda,9655,1474,Unfurnished,2,Hyderabad,...,17,0,1,0,0,1,19310,0,1,1600
2,99943,3,BHK,Apartment,Jodhpur Park,23699,1837,Semi-Furnished,2,Kolkata,...,10,1,1,1,1,0,118495,0,1,3100
3,113926,1,BHK,Apartment,Chembur,6306,606,Unfurnished,1,Mumbai,...,18,0,0,0,0,0,37836,0,1,300
4,185438,1,BHK,Studio Apartment,Kalewadi Pandhapur Road,12008,498,Semi-Furnished,3,Pune,...,14,0,0,1,1,0,72048,0,1,0


We need to predict the price

In [63]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Property_ID,134683.0,96036.100777,55565.228125,2.0,47940.0,95950.0,144194.5,192405.0
room,134683.0,2.029677,0.937308,1.0,1.0,2.0,3.0,5.0
price,134683.0,36690.033894,62620.364025,1583.0,12035.5,20856.0,36014.0,799325.0
area,134683.0,1480.388490,1412.464718,81.0,759.0,1114.0,1580.0,13942.0
bathroom,134683.0,2.040488,0.867065,1.0,1.0,2.0,2.0,5.0
parking_spaces,134683.0,0.534388,0.498818,0.0,0.0,1.0,1.0,1.0
floor,134683.0,9.163087,5.957549,0.0,4.0,9.0,15.0,19.0
pet_friendly,134683.0,0.527602,0.499239,0.0,0.0,1.0,1.0,1.0
power_backup,134683.0,0.337051,0.472704,0.0,0.0,0.0,1.0,1.0
washing_machine,134683.0,0.472561,0.499248,0.0,0.0,0.0,1.0,1.0


In [64]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134683 entries, 0 to 134682
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Property_ID       134683 non-null  int64 
 1   room              134683 non-null  int64 
 2   layout_type       134683 non-null  object
 3   property_type     134683 non-null  object
 4   locality          134683 non-null  object
 5   price             134683 non-null  int64 
 6   area              134683 non-null  int64 
 7   furnish_type      134683 non-null  object
 8   bathroom          134683 non-null  int64 
 9   city              134683 non-null  object
 10  parking_spaces    134683 non-null  int64 
 11  floor             134683 non-null  int64 
 12  pet_friendly      134683 non-null  int64 
 13  power_backup      134683 non-null  int64 
 14  washing_machine   134683 non-null  int64 
 15  air_conditioner   134683 non-null  int64 
 16  geyser/solar      134683 non-null  int

The data have no null values, Uff... It eases our work somehow..

##  EDA

In [65]:
# plt.rcParams['figure.figsize'] = [28, 10]
# plt.rcParams['figure.dpi'] = 200
# plt.rcParams['axes.labelsize'] = 20
# plt.rcParams['xtick.labelsize'] = 20
# plt.rcParams['ytick.labelsize'] = 20

In [66]:
# print(train_df["room"].value_counts())
# sns.boxenplot(x=train_df["room"], y=train_df["price"])
# plt.show()

The boxplots were soo conjusted that we can't deduce anything from them.

In [67]:
# print(train_df["layout_type"].value_counts())
# sns.boxenplot(x=train_df["layout_type"], y=train_df["price"])
# plt.show()

Hmm... Even this isn't very informative. May be I'm not using a right method to plot. Let me try another method.

In [68]:
X_train, y_train = train_df.drop("price", axis=1), train_df["price"]

In [69]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error

In [70]:
X_train["layout_type"] = LabelEncoder().fit_transform(X_train["layout_type"])
X_train["property_type"] = LabelEncoder().fit_transform(X_train["property_type"])
X_train["locality"] = LabelEncoder().fit_transform(X_train["locality"])
X_train["furnish_type"] = LabelEncoder().fit_transform(X_train["furnish_type"])
X_train["city"] = LabelEncoder().fit_transform(X_train["city"])

In [71]:
X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=0.25)

In [72]:
test_df["layout_type"] = LabelEncoder().fit_transform(test_df["layout_type"])
test_df["property_type"] = LabelEncoder(
).fit_transform(test_df["property_type"])
test_df["locality"] = LabelEncoder().fit_transform(test_df["locality"])
test_df["furnish_type"] = LabelEncoder().fit_transform(test_df["furnish_type"])
test_df["city"] = LabelEncoder().fit_transform(test_df["city"])

In [73]:
test = test_df.drop("price", axis=1)

In [74]:
import optuna

In [75]:
# def objective(trial, X_train = X_train, y_train = y_train, X_test = X_test, y_test = y_test):
    
# #     train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
#     param = {
#         'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
#         'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
#         'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
#         'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
#         'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
#         'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
#         'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
#         'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
#         'random_state': trial.suggest_categorical('random_state', [2020]),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
#     }
#     model = xgb.XGBRegressor(**param)  
    
#     model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=100,verbose=False)
    
#     preds = model.predict(X_test)
    
#     rmse = mean_squared_error(y_test, preds,squared=False)
    
#     return rmse

In [76]:
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=100)
# print('Number of finished trials:', len(study.trials))
# print('Best trial:', study.best_trial.params)

In [77]:
# best_params = study.best_trial.params
best_params = {
    'lambda': 0.0028529452269855336,
    'alpha': 0.003297186319062336,
    'colsample_bytree': 1.0,
    'tree_method': 'gpu_hist',
    'subsample': 0.8,
    'learning_rate': 0.016,
    'n_estimators': 9205,
    'max_depth': 7,
    'random_state': 2020,
    'min_child_weight': 299
}

model_xgb = xgb.XGBRegressor(**best_params)

In [78]:
if not os.path.exists("xgb_finetuned.json"):
    model_xgb.fit(X_train, y_train)
    model_xgb.save_model("xgb_finetuned.json")
else:
    print("Loading Model...")
    model_xgb.load_model("xgb_finetuned.json")


Loading Model...


In [79]:
mean_squared_error(y_test, model_xgb.predict(X_test), squared=False)

24528.97746837569

In [80]:
from sklearn.linear_model import LinearRegression

model_lr = LinearRegression()

model_lr.fit(X_train, y_train)


LinearRegression()

In [81]:
mean_squared_error(y_test, model_lr.predict(X_test), squared=False)

27475.096198152634

In [82]:
# from catboost import CatBoostRegressor

# def catboost_objective(trial,X_train = X_train, y_train = y_train, X_test = X_test, y_test = y_test):
    
# #     train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
#     param = {
#         'loss_function': 'RMSE',
#         'task_type': 'GPU',
#         'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
#         'max_bin': trial.suggest_int('max_bin', 200, 400),
#         #'rsm': trial.suggest_uniform('rsm', 0.3, 1.0),
# #         'subsample': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
#         'learning_rate': trial.suggest_uniform('learning_rate', 0.006, 0.018),
#         'n_estimators':  25000,
#         'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15]),
#         'random_state': trial.suggest_categorical('random_state', [2020]),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 300),
#     }
#     model = CatBoostRegressor(**param)  
    
#     model.fit(X_train,y_train,eval_set=[(X_test, y_test)],early_stopping_rounds=200,verbose=False)
    
#     preds = model.predict(X_test)
    
#     rmse = mean_squared_error(y_test, preds,squared=False)
    
#     return rmse

In [83]:
# study = optuna.create_study(direction='minimize')
# study.optimize(catboost_objective, n_trials=30)
# print('Number of finished trials:', len(study.trials))
# print('Best trial:', study.best_trial.params)

In [84]:
# best_params = study.best_trial.params
best_params = {
    'l2_leaf_reg': 2.6330844914053784,
    'max_bin': 366,
    'learning_rate': 0.011929381903465954,
    'max_depth': 7,
    'random_state': 2020,
    'min_data_in_leaf': 185
}

model_catboost = catboost.CatBoostRegressor(**best_params)

model_catboost.fit(X_train, y_train)


0:	learn: 62252.8327499	total: 56.1ms	remaining: 56s
1:	learn: 61684.3650312	total: 63.9ms	remaining: 31.9s
2:	learn: 61118.2718948	total: 73.5ms	remaining: 24.4s
3:	learn: 60559.0351294	total: 84.2ms	remaining: 21s
4:	learn: 60005.7838941	total: 93.2ms	remaining: 18.6s
5:	learn: 59463.2767051	total: 101ms	remaining: 16.7s
6:	learn: 58916.9704848	total: 107ms	remaining: 15.2s
7:	learn: 58390.1984980	total: 114ms	remaining: 14.1s
8:	learn: 57861.0385271	total: 122ms	remaining: 13.5s
9:	learn: 57346.7410570	total: 128ms	remaining: 12.7s
10:	learn: 56845.4298437	total: 135ms	remaining: 12.1s
11:	learn: 56336.5357581	total: 142ms	remaining: 11.7s
12:	learn: 55844.6916478	total: 149ms	remaining: 11.3s
13:	learn: 55379.7602294	total: 157ms	remaining: 11s
14:	learn: 54908.3982363	total: 165ms	remaining: 10.8s
15:	learn: 54445.5423737	total: 171ms	remaining: 10.5s
16:	learn: 53979.6022248	total: 177ms	remaining: 10.2s
17:	learn: 53522.9832271	total: 184ms	remaining: 10s
18:	learn: 53077.541084

153:	learn: 28626.8692473	total: 1.07s	remaining: 5.87s
154:	learn: 28587.8962572	total: 1.07s	remaining: 5.86s
155:	learn: 28550.0012104	total: 1.08s	remaining: 5.85s
156:	learn: 28510.1955015	total: 1.09s	remaining: 5.85s
157:	learn: 28471.9229808	total: 1.09s	remaining: 5.84s
158:	learn: 28434.8903049	total: 1.1s	remaining: 5.83s
159:	learn: 28399.3643304	total: 1.11s	remaining: 5.82s
160:	learn: 28364.9274467	total: 1.11s	remaining: 5.81s
161:	learn: 28330.3104453	total: 1.12s	remaining: 5.8s
162:	learn: 28297.8013163	total: 1.13s	remaining: 5.79s
163:	learn: 28264.5716427	total: 1.13s	remaining: 5.77s
164:	learn: 28231.8526657	total: 1.14s	remaining: 5.76s
165:	learn: 28199.1047480	total: 1.14s	remaining: 5.75s
166:	learn: 28167.2437824	total: 1.15s	remaining: 5.74s
167:	learn: 28135.0813993	total: 1.16s	remaining: 5.73s
168:	learn: 28104.9217126	total: 1.16s	remaining: 5.72s
169:	learn: 28075.1636416	total: 1.17s	remaining: 5.71s
170:	learn: 28046.1100871	total: 1.18s	remaining: 

300:	learn: 26616.9244529	total: 2.08s	remaining: 4.83s
301:	learn: 26612.5571001	total: 2.08s	remaining: 4.82s
302:	learn: 26608.6136785	total: 2.09s	remaining: 4.81s
303:	learn: 26604.5027030	total: 2.1s	remaining: 4.81s
304:	learn: 26601.2121292	total: 2.11s	remaining: 4.8s
305:	learn: 26597.8376109	total: 2.11s	remaining: 4.79s
306:	learn: 26594.1248582	total: 2.12s	remaining: 4.79s
307:	learn: 26590.5776617	total: 2.13s	remaining: 4.78s
308:	learn: 26587.1477370	total: 2.13s	remaining: 4.77s
309:	learn: 26584.1383514	total: 2.14s	remaining: 4.76s
310:	learn: 26580.9120061	total: 2.15s	remaining: 4.75s
311:	learn: 26577.3030163	total: 2.15s	remaining: 4.74s
312:	learn: 26574.3832095	total: 2.16s	remaining: 4.74s
313:	learn: 26570.4330746	total: 2.16s	remaining: 4.73s
314:	learn: 26567.5054548	total: 2.17s	remaining: 4.72s
315:	learn: 26565.7983272	total: 2.17s	remaining: 4.7s
316:	learn: 26562.5414492	total: 2.18s	remaining: 4.69s
317:	learn: 26560.0369773	total: 2.19s	remaining: 4

455:	learn: 26200.7065094	total: 3.3s	remaining: 3.93s
456:	learn: 26199.1121062	total: 3.31s	remaining: 3.93s
457:	learn: 26196.9028800	total: 3.32s	remaining: 3.93s
458:	learn: 26195.3628349	total: 3.33s	remaining: 3.92s
459:	learn: 26192.6272024	total: 3.34s	remaining: 3.92s
460:	learn: 26190.7080238	total: 3.35s	remaining: 3.92s
461:	learn: 26189.2199418	total: 3.36s	remaining: 3.92s
462:	learn: 26187.9086018	total: 3.37s	remaining: 3.91s
463:	learn: 26184.9357446	total: 3.38s	remaining: 3.9s
464:	learn: 26182.6656012	total: 3.39s	remaining: 3.9s
465:	learn: 26181.1251123	total: 3.4s	remaining: 3.89s
466:	learn: 26177.8289875	total: 3.4s	remaining: 3.88s
467:	learn: 26176.3285719	total: 3.41s	remaining: 3.88s
468:	learn: 26174.0068744	total: 3.42s	remaining: 3.87s
469:	learn: 26172.5716368	total: 3.42s	remaining: 3.86s
470:	learn: 26170.1637275	total: 3.43s	remaining: 3.85s
471:	learn: 26167.0390066	total: 3.44s	remaining: 3.84s
472:	learn: 26165.0316712	total: 3.44s	remaining: 3.8

623:	learn: 25870.7570117	total: 4.51s	remaining: 2.71s
624:	learn: 25868.9705597	total: 4.52s	remaining: 2.71s
625:	learn: 25867.5847681	total: 4.53s	remaining: 2.7s
626:	learn: 25866.1292977	total: 4.54s	remaining: 2.7s
627:	learn: 25862.9749574	total: 4.55s	remaining: 2.69s
628:	learn: 25860.8580290	total: 4.56s	remaining: 2.69s
629:	learn: 25859.4112818	total: 4.57s	remaining: 2.68s
630:	learn: 25856.1683888	total: 4.58s	remaining: 2.67s
631:	learn: 25853.8951458	total: 4.58s	remaining: 2.67s
632:	learn: 25851.0161243	total: 4.59s	remaining: 2.66s
633:	learn: 25848.5958628	total: 4.6s	remaining: 2.66s
634:	learn: 25846.8245279	total: 4.61s	remaining: 2.65s
635:	learn: 25844.5937305	total: 4.62s	remaining: 2.64s
636:	learn: 25842.2597560	total: 4.63s	remaining: 2.64s
637:	learn: 25841.0324435	total: 4.64s	remaining: 2.63s
638:	learn: 25838.7715233	total: 4.65s	remaining: 2.62s
639:	learn: 25836.7645896	total: 4.66s	remaining: 2.62s
640:	learn: 25835.8131580	total: 4.66s	remaining: 2

778:	learn: 25561.2125876	total: 6.39s	remaining: 1.81s
779:	learn: 25558.9031692	total: 6.4s	remaining: 1.8s
780:	learn: 25556.4004842	total: 6.41s	remaining: 1.8s
781:	learn: 25552.8100327	total: 6.42s	remaining: 1.79s
782:	learn: 25550.2769653	total: 6.44s	remaining: 1.78s
783:	learn: 25548.1209076	total: 6.45s	remaining: 1.78s
784:	learn: 25546.6648280	total: 6.46s	remaining: 1.77s
785:	learn: 25545.1488838	total: 6.47s	remaining: 1.76s
786:	learn: 25542.4931711	total: 6.48s	remaining: 1.75s
787:	learn: 25540.7259322	total: 6.5s	remaining: 1.75s
788:	learn: 25539.5962867	total: 6.5s	remaining: 1.74s
789:	learn: 25538.3448623	total: 6.51s	remaining: 1.73s
790:	learn: 25536.5986925	total: 6.52s	remaining: 1.72s
791:	learn: 25534.8931676	total: 6.54s	remaining: 1.72s
792:	learn: 25534.0812383	total: 6.54s	remaining: 1.71s
793:	learn: 25531.5177648	total: 6.55s	remaining: 1.7s
794:	learn: 25530.3157327	total: 6.56s	remaining: 1.69s
795:	learn: 25528.4246034	total: 6.57s	remaining: 1.68

933:	learn: 25261.7020084	total: 8.03s	remaining: 567ms
934:	learn: 25259.8961299	total: 8.06s	remaining: 560ms
935:	learn: 25257.9414495	total: 8.07s	remaining: 552ms
936:	learn: 25256.0898832	total: 8.08s	remaining: 543ms
937:	learn: 25254.5940663	total: 8.09s	remaining: 535ms
938:	learn: 25252.0495511	total: 8.1s	remaining: 526ms
939:	learn: 25249.8782835	total: 8.11s	remaining: 518ms
940:	learn: 25248.5401389	total: 8.12s	remaining: 509ms
941:	learn: 25246.9571677	total: 8.13s	remaining: 500ms
942:	learn: 25245.9843086	total: 8.14s	remaining: 492ms
943:	learn: 25244.9398035	total: 8.14s	remaining: 483ms
944:	learn: 25244.1786143	total: 8.15s	remaining: 475ms
945:	learn: 25242.6982779	total: 8.16s	remaining: 466ms
946:	learn: 25241.6431557	total: 8.17s	remaining: 457ms
947:	learn: 25239.9124258	total: 8.18s	remaining: 449ms
948:	learn: 25237.1107153	total: 8.19s	remaining: 440ms
949:	learn: 25235.6889846	total: 8.2s	remaining: 432ms
950:	learn: 25233.7768476	total: 8.21s	remaining: 

In [85]:
mean_squared_error(y_test, model_catboost.predict(X_test), squared=False)

26877.45991637415

In [86]:
from sklearn.linear_model import Lasso

model_lasso = Lasso()

model_lasso.fit(X_train, y_train)

mean_squared_error(y_test, model_lasso.predict(X_test), squared=False)

27475.09474180301

In [87]:
model_base_xgb = xgb.XGBRFRegressor()
model_base_xgb.fit(X_train, y_train)
mean_squared_error(y_test, model_base_xgb.predict(X_test), squared=False)

26934.649281506532

In [88]:
mean_squared_error(
    y_test,
    (model_xgb.predict(X_test) + model_lr.predict(X_test) +
     model_catboost.predict(X_test) + model_lasso.predict(X_test) +
     model_base_xgb.predict(X_test)) / 5,
    squared=False)

26097.105917775294

In [89]:
mean_squared_error(y_test,
                   (model_xgb.predict(X_test) + model_lr.predict(X_test) +
                    model_catboost.predict(X_test) + model_lasso.predict(X_test)) / 4,
                   squared=False)

25979.736093324504

In [90]:
mean_squared_error(y_test,
                   (model_xgb.predict(X_test) + model_lr.predict(X_test) +
                    model_catboost.predict(X_test)) / 3,
                   squared=False)

25677.141221355763

In [91]:
mean_squared_error(y_test, model_lasso.predict(X_test), squared=False)

27475.09474180301

In [92]:
mean_squared_error(y_test,
                    0.30 * model_xgb.predict(X_test) + 
                    0.10 * model_lr.predict(X_test) +
                    0.60 * model_catboost.predict(X_test),
                   squared=False)

25835.563384103905

In [93]:
mean_squared_error(y_test,
                    0.30 * model_xgb.predict(X_test) + 
                    0.20 * model_lr.predict(X_test) +
                    0.50 * model_catboost.predict(X_test),
                   squared=False)

25788.165487938964

In [109]:
mean_squared_error(y_test,
                    0.25 * model_xgb.predict(X_test) + 
                    0.35 * model_lr.predict(X_test) +
                    0.40 * model_catboost.predict(X_test),
                   squared=False)

25923.426141719854

In [104]:
mean_squared_error(y_test,
                   0.10 * model_xgb.predict(X_test) + 
                   0.15 * model_lr.predict(X_test) +
                   0.10 * model_catboost.predict(X_test) +
                   0.15 * model_lasso.predict(X_test) +
                   0.50 * model_base_xgb.predict(X_test),
                   squared=False)

26436.94219214358

In [96]:
mean_squared_error(
    y_test,
    (
     model_catboost.predict(X_test) + model_lasso.predict(X_test) +
     model_base_xgb.predict(X_test)) / 3,
    squared=False)

26762.666508742586

In [97]:
# import torch

# class CustomDataset(torch.utils.data.Dataset):
#     def __init__(self, X, y):
#         self.X = np.array(X)
#         self.y = np.array(y).reshape(-1, 1)
        
#     def __len__(self):
#         return len(self.X)
    
#     def __getitem__(self, idx):
#         X = self.X[idx, :]
#         y = self.y[idx, :]
        
#         return (X, y)
        

In [98]:
# train_dataset = CustomDataset(X_train, y_train)
# training_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

In [99]:
# import tqdm
# class LinearRegression(torch.nn.Module):
#     def __init__(self, in_features):
#         super().__init__()
#         self.linear = torch.nn.Linear(in_features, 1)
        
#     def forward(self, x):
#         return self.linear(x)
        
# model = LinearRegression(X_train.shape[1]) # Initialize the model

# loss_fn = torch.nn.MSELoss()
# # loss_fn = torch.sqrt(criterion(x, y))
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # Define the optimizer

In [100]:
# prev_loss = 0
# for epoch in range(10000):
#     model.train() # Set the model to training mode
    
#     for (X_train, y_train) in training_loader
#     y_pred = model(X_train) # Evaluate the model on the training data
#     loss = loss_fn(y_pred, y_train) # Calculate the loss


#     optimizer.zero_grad() # Reset the gradients for optimizer
#     loss.backward() # Calculate the gradients for loss_fn wrt model parameters
#     optimizer.step() # Update the model parameters

#     model.eval() # Set the model to evaluation mode
#     y_pred_test = model(X_test) # Evaluate the model on the test data
#     loss_test = torch.sqrt(loss_fn(y_pred_test, y_test)) # Calculate the loss


    
#     prev_loss = loss.item()
#     if epoch % 100 == 0:
#         print(f"Epoch: {epoch}, loss: {loss.item()}, loss_test: {loss_test.item()}")

In [101]:
# prev_loss = 0
# for epoch in range(2):
#     model.train() # Set the model to training mode

#     with tqdm.tqdm(training_loader) as bar:
#         for batch_x, batch_y in bar:
#             batch_x = batch_x.to("cuda")
#             batch_y = batch_y.to("cuda")
#             y_pred = model(batch_x.float()) # Evaluate the model on the training data
#             loss = loss_fn(y_pred.float(), batch_y.float()) # Calculate the loss

#             # Early stopping
#             # if (abs(loss.item() - prev_loss)) < 1e-6:
#             #     print(f"Converged after {epoch} epochs")
#             #     break
#             # print("Model params: ", model.state_dict())
#             optimizer.zero_grad() # Reset the gradients for optimizer
#             loss.backward() # Calculate the gradients for loss_fn wrt model parameters
#             optimizer.step() # Update the model parameters

#             # non_linear_model.eval() # Set the model to evaluation mode
#             # y_pred_test = non_linear_model(X_test) # Evaluate the model on the test data
#             # loss_test = loss_fn(y_pred_test, y_test) # Calculate the loss

#     prev_loss = loss.item()
#     # print(f"Epoch: {epoch}, loss: {loss.item()}, loss_test: {loss_test.item()}")
#     print(f"Epoch: {epoch}, loss: {loss.item()}")

        

In [110]:
submission = pd.read_csv("submission.csv")
submission['price'] = (0.25 * model_xgb.predict(test) + 
                    0.35 * model_lr.predict(test) +
                    0.40 * model_catboost.predict(test))
submission.to_csv("final_submission.csv", index=False)